In [138]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [139]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [140]:
LOCAL = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [141]:
%%capture pwd
!pwd

In [142]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('the-shift-to-working-from-home-how-has-it-affected-productivity', 300, 500)

# Fig 1

## a

In [143]:
df = pd.read_csv("raw/1a.csv")[1:]
df=df[[df.columns[i] for i in range(len(df.columns)) if i in [1,3,5]]].astype(float)/100
df.columns=['total','within','between']
df['Q']=['2020 Q2','2020 Q3','2020 Q4','2021 Q1','2021 Q2','2021 Q3','2021 Q4','2022+']

In [144]:
f = "fig1a_labor_productivity"
f1a = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1a.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1a = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,total,within,between,Q
1,0.099888,0.114045,-0.012584,2020 Q2
2,0.024382,0.045618,-0.020449,2020 Q3
3,0.005506,0.044831,-0.037753,2020 Q4
4,0.044045,0.073933,-0.029101,2021 Q1
5,0.000000,0.025955,-0.024382,2021 Q2


In [145]:
base = alt.Chart(f1a).encode(
    x=alt.X(
        "Q:N",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
bars1 = base.mark_bar(color=colors["eco-turquiose"],width=25).encode(
    y=alt.Y(
        "within:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            title="Percentage impact of Covid-19 on labor productivity per hour",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            grid=True,
            # titleAnchor="start",
            # labelColor=colors["eco-gray"],
            # titleColor=colors["eco-gray"],
            # tickColor=colors["eco-gray"],
            # domainColor=colors["eco-gray"],
            # titleFontSize=10,
            # titleFontWeight="normal",
            # ticks=False,
            # labelAlign="left",
            # labelBaseline="middle",
            # labelPadding=-5,
            # labelOffset=-10,
            # titleX=23,
            # titleY=22,
            # titleBaseline="bottom",
            # titleAngle=0,
            # titleAlign="left",
            tickCount=7,
            format=".0%",
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)
bars2 = base.mark_bar(color=colors["eco-light-blue"],width=25).encode(
    y=alt.Y(
        "between:Q",
        sort=[]
    )
)
line=base.mark_line(color=colors["eco-mid-blue"]).encode(
    y=alt.Y(
        "t:Q",
        sort=[]
    )
).transform_calculate(t='datum.within+datum.between')
points=line.mark_point(color=colors["eco-mid-blue"],fill=colors["eco-mid-blue"],opacity=1)
zeroline=base.mark_line(color=colors["eco-gray"],opacity=0.5).encode(
    y=alt.Y(
        "z:Q",
        sort=[]
    )
).transform_calculate(z='0')
labels=alt.Chart(pd.DataFrame(
    [
        {'x':'2020 Q2','y':0.105,'t':'Between-firm'},
        {'x':'2020 Q4','y':-0.036,'t':'Within-firm'},
        {'x':'2021 Q3','y':-0.035,'t':'Total'}
    ])).mark_text(dx=20,align='left').encode(
    x="x:N",
    y="y:Q",
    text='t:N',
    color=alt.Color('t:N',scale=alt.Scale(
        range=[colors['eco-turquiose'],colors['eco-mid-blue'],colors['eco-light-blue']]),
        legend=None
    )
)
layer1 = (
    ((zeroline+bars1+bars2+line+points+labels).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "w").write(readme)
layer1

WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
Fontconfig error: Cannot load default config file


alt.LayerChart(...)

## b

In [146]:
df = pd.read_csv("raw/1b.csv")[1:]
df=df[[df.columns[i] for i in range(len(df.columns)) if i in [1,3,5]]].astype(float)/100
df.columns=['total','within','between']
df['Q']=['2020 Q2','2020 Q3','2020 Q4','2021 Q1','2021 Q2','2021 Q3','2021 Q4','2022+']

In [147]:
f = "fig1b_TFP"
f1b = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1b.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1b = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,total,within,between,Q
1,-0.023162,0.068316,-0.094695,2020 Q2
2,-0.042709,0.013264,-0.057794,2020 Q3
3,-0.044239,0.018037,-0.064190,2020 Q4
4,-0.019508,0.035474,-0.056978,2021 Q1
5,-0.012291,0.024698,-0.037601,2021 Q2


In [148]:
base = alt.Chart(f1b).encode(
    x=alt.X(
        "Q:N",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
bars1 = base.mark_bar(color=colors["eco-turquiose"],width=25).encode(
    y=alt.Y(
        "within:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            title="Percentage impact of Covid-19 on TFP",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            grid=True,
            # titleAnchor="start",
            # labelColor=colors["eco-gray"],
            # titleColor=colors["eco-gray"],
            # tickColor=colors["eco-gray"],
            # domainColor=colors["eco-gray"],
            # titleFontSize=10,
            # titleFontWeight="normal",
            # ticks=False,
            # labelAlign="left",
            # labelBaseline="middle",
            # labelPadding=-5,
            # labelOffset=-10,
            # titleX=23,
            # titleY=22,
            # titleBaseline="bottom",
            # titleAngle=0,
            # titleAlign="left",
            tickCount=7,
            format=".0%",
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)
bars2 = base.mark_bar(color=colors["eco-light-blue"],width=25).encode(
    y=alt.Y(
        "between:Q",
        sort=[]
    )
)
line=base.mark_line(color=colors["eco-mid-blue"]).encode(
    y=alt.Y(
        "t:Q",
        sort=[]
    )
).transform_calculate(t='datum.within+datum.between')
points=line.mark_point(color=colors["eco-mid-blue"],fill=colors["eco-mid-blue"],opacity=1)
zeroline=base.mark_line(color=colors["eco-gray"],opacity=0.5).encode(
    y=alt.Y(
        "z:Q",
        sort=[]
    )
).transform_calculate(z='0')
labels=alt.Chart(pd.DataFrame(
    [
        {'x':'2020 Q2','y':0.065,'t':'Between-firm'},
        {'x':'2020 Q2','y':-0.075,'t':'Within-firm'},
        {'x':'2021 Q3','y':-0.037,'t':'Total'}
    ])).mark_text(dx=20,align='left').encode(
    x="x:N",
    y="y:Q",
    text='t:N',
    color=alt.Color('t:N',scale=alt.Scale(
        range=[colors['eco-turquiose'],colors['eco-mid-blue'],colors['eco-light-blue']]),
        legend=None
    )
)
layer1 = (
    ((zeroline+bars1+bars2+line+points+labels).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
Fontconfig error: Cannot load default config file


alt.LayerChart(...)

# Fig 2

## a

In [149]:
df = pd.read_csv("raw/2a.csv",header=None)[[1]]
df.columns=['hours']
df['month']=range(-11,6)

In [150]:
f = "fig2a_time"
f2a = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2a.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2a = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,hours,month
0,5.103448,-11
1,4.818966,-10
2,4.961207,-9
3,5.116379,-8
4,4.586207,-7


In [151]:
base = alt.Chart(f2a).encode(
    x=alt.X(
        "month:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="month",
            titleY=-15,
            titleX=288,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
line = base.mark_line(color=colors["eco-turquiose"],interpolate="monotone").encode(
    y=alt.Y(
        "hours:Q",
        sort=[],
        axis=alt.Axis(
#             grid=False,
            title="Time worked per working day",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            format='.0f',
            tickCount=4,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        scale=alt.Scale(domain=[4,8]),
    )
)
k=str(0.015)
area=base.mark_area(color=colors["eco-turquiose"],opacity=0.3,interpolate="monotone").encode(
    y='y:Q',y2='y2:Q'
).transform_calculate(y='datum.hours*(1-'+k+')',y2='datum.hours*(1+'+k+')')
vertical=alt.Chart(pd.DataFrame([{'x':0}])).mark_rule(strokeDash=[5,5],color=colors['eco-gray'],opacity=0.5).encode(x='x:Q')
layer1 = (
    ((line+area+vertical).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.LayerChart(...)

## b

In [152]:
df = pd.read_csv("raw/2b.csv",header=None)[[1]]
df.columns=['output']
df['month']=range(-11,6)
k=0.0025
df['y']=df['output']*(1-k)
df['y2']=df['output']*(1+k)
df.loc[1,'y']=102
df.loc[1,'y2']=103

In [153]:
f = "fig2b_output"
f2b = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2b.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2b = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,output,month,y,y2
0,100.220209,-11,99.969659,100.470760
1,102.458460,-10,102.000000,103.000000
2,100.662191,-9,100.410536,100.913847
3,100.766822,-8,100.514905,101.018739
4,100.871414,-7,100.619236,101.123593


In [154]:
base = alt.Chart(f2b).encode(
    x=alt.X(
        "month:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="month",
            titleY=-15,
            titleX=288,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
line = base.mark_line(color=colors["eco-turquiose"],interpolate="monotone").encode(
    y=alt.Y(
        "output:Q",
        sort=[],
        axis=alt.Axis(
#             grid=False,
            title="Output by month",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            format='.0f',
            tickCount=4,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        scale=alt.Scale(domain=[99,103]),
    )
)
area=base.mark_area(color=colors["eco-turquiose"],opacity=0.3,interpolate="monotone").encode(
    y='y:Q',y2='y2:Q'
)
vertical=alt.Chart(pd.DataFrame([{'x':0}])).mark_rule(strokeDash=[5,5],color=colors['eco-gray'],opacity=0.5).encode(x='x:Q')
layer1 = (
    ((line+area+vertical).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

UnboundLocalError: local variable 'stderr' referenced before assignment

## c

In [ ]:
df = pd.read_csv("raw/2c.csv")[1:][['Unnamed: 11','Unnamed: 15']].astype(float)
df.columns=['output','y2']
df['month']=range(-11,6)
df['y']=2*df['output']-df['y2']

In [ ]:
f = "fig2c_productivity"
f2c = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2c.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2c = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

In [ ]:
base = alt.Chart(f2c).encode(
    x=alt.X(
        "month:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="month",
            titleY=-15,
            titleX=288,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
line = base.mark_line(color=colors["eco-turquiose"],interpolate="monotone").encode(
    y=alt.Y(
        "output:Q",
        sort=[],
        axis=alt.Axis(
#             grid=False,
            title="Productivity",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            format='.1f',
            tickCount=4,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        scale=alt.Scale(domain=[0.9,1.7]),
    )
)
area=base.mark_area(color=colors["eco-turquiose"],opacity=0.3,interpolate="monotone").encode(
    y='y:Q',y2='y2:Q'
)
vertical=alt.Chart(pd.DataFrame([{'x':0}])).mark_rule(strokeDash=[5,5],color=colors['eco-gray'],opacity=0.5).encode(x='x:Q')
layer1 = (
    ((line+area+vertical).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

# Fig 3

In [ ]:
df = pd.DataFrame([{'x':'Much more, >35%','y':13.2},
                  {'x':'Substantially more, 15-25%','y':8.3},
                   {'x':'More, <15%','y':18.2},
                   {'x':'About the same','y':45},
                   {'x':'Less, <15%','y':5.7},
                   {'x':'Substantially less, 15-25%','y':3.8},
                   {'x':'Much less, >35%','y':5.7}
                  ])
df['y']=df['y']/100.0

In [ ]:
f = "fig3_efficiency"
f3 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f3.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f3 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

In [ ]:
base = alt.Chart(f3).encode(
    y=alt.Y('x:N',sort=[],
           axis=alt.Axis(
#             grid=False,
            title="",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=4,
               tickSize=0,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
           ))
bars = base.mark_bar(color=colors["eco-light-blue"],height=25).encode(
    x=alt.X(
        "y:Q",
        sort=[],
        axis=alt.Axis(format='.0%',
#                      grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=288,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
                     )
    )
)
layer1 = (
    ((bars).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1